For this iteration, I removed my model and only used my preparation of the dataset to start, and tried to then plug that into his code. I wanted to see if it at least gave me a different error than v3, but it seems to hit the same problem.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error

In [2]:
dataset_train = pd.read_csv('MSFT 5-15 to 5-19.csv')
print(dataset_train.head())
print(dataset_train.info())
print(dataset_train.isna().sum())
training_set = dataset_train.iloc[:, 1:2].values

         Date       Open       High        Low      Close  Adj Close    Volume
0  2015-05-04  48.369999  48.869999  48.180000  48.240002  43.497272  34039500
1  2015-05-05  47.820000  48.160000  47.310001  47.599998  42.920189  50369200
2  2015-05-06  47.570000  47.770000  46.020000  46.279999  41.729969  52433000
3  2015-05-07  46.270000  47.090000  46.160000  46.700001  42.108677  32971700
4  2015-05-08  47.549999  47.980000  47.520000  47.750000  43.055447  35364900
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 7 columns):
Date         1007 non-null object
Open         1007 non-null float64
High         1007 non-null float64
Low          1007 non-null float64
Close        1007 non-null float64
Adj Close    1007 non-null float64
Volume       1007 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 55.2+ KB
None
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype

In [3]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [4]:
X_train = []
y_train = []
for i in range(60, 1007):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [6]:
dataset_test = pd.read_csv('MSFT.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values

In [7]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

In [8]:
X_test = []
for i in range(60, 312):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [9]:
#n_nodes = [100]
n_epochs = [50, 100]
n_batch = [32, 64]
#configs = [n_epochs, n_batch]
#configs

In [10]:
def model_fit(X_train, y_train):
    model = Sequential()
    model.add(LSTM(units = 100, return_sequences = True, input_shape=(X_train.shape[1],1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 100, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 100, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = 100))
    model.add(Dropout(0.2))
    model.add(Dense(units = 1))
    model.compile(loss='mse', optimizer='adam')
    
    model.fit(X_train, y_train, epochs = n_epochs, batch_size = n_batch)
    return model

In [11]:
def walk_forward_validation(X_train, X_test, cfg):
    predictions = list()
    # split dataset
    # fit model
    model = model_fit(X_train, cfg)
    # seed history with training dataset
    history = [x for x in X_train]
    # step over each time-step in the test set
    for i in range(len(X_test)):
        # fit model and make forecast for history
        yhat = model_predict(model, history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = np.sqrt(mean_squared_error(X_test, predictions))
    print(' > %.3f' % error)
    return error

In [12]:
def repeat_evaluate(X_train, config, X_test, n_repeats=10):
    # convert config to a key
    key = str(config)
    # fit and evaluate the model n times
    scores = [walk_forward_validation(X_train, X_test, config) for _ in range(n_repeats)]
    # summarize score
    result = mean(scores)
    print('> Model[%s] %.3f' % (key, result))
    return (key, result)

In [13]:
def grid_search(X_train, cfg_list, X_test):
    # evaluate configs
    scores = [repeat_evaluate(X_train, cfg, X_test) for cfg in cfg_list]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    return scores

In [14]:
def model_configs():
    # define scope of configs
    #n_nodes = [100]
    n_epochs = [50, 100]
    n_batch = [32, 64]
    # create configs
    configs = list()
    for i in n_epochs:
        for j in n_batch:    
            cfg = [i, j]
            configs.append(cfg)
    print('Total configs: %d' % len(configs))
    return configs

In [15]:
cfg_list = model_configs()
# grid search
scores = grid_search(X_train, cfg_list, X_test)
print('done')
# list top 10 configs
for cfg, error in scores[:5]:
    print(cfg, error)

Total configs: 4


ValueError: Input arrays should have the same number of samples as target arrays. Found 947 input samples and 2 target samples.